In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../')
from pose_graph_partitioning.pose_graph import *
from pose_graph_partitioning.pose_graph_partitioning import *
from simulate_utils import ATE
from subprocess import PIPE, Popen
import copy

In [2]:
def call_d2pgo(input_data, output_data, agent_num, params, cmd="roslaunch d2pgo d2pgo_test_multi.launch "):
    command = f"{cmd} g2o_path:={input_data} output_path:={output_data} agent_num:={agent_num} "
    for k, v in params:
        command += f"{k}:={v[0]} "
    p = Popen(command, shell=True, stdout=PIPE, stderr=PIPE)
    stdout, stderr = p.communicate()
    # return stdout, stderr
    # print(command)
    # print(stdout.decode("utf-8"))
    return

def check_param_unique(params):
    cnt = 0
    for k, v in params:
        if len(v) > 1:
            return False, cnt
        cnt += 1
    return True, -1 

def print_res_to_table(res, sort_index="ATE_T"):
    import tabulate
    output_table = []
    #Add title
    if len(res) == 0:
        return []
    sort_values = []
    for row in res:
        output_table.append([])
        for k in row:
            v = row[k]
            if isinstance(v, np.ndarray) or isinstance(v, list):
                output_table[-1].append(v[0])
            else:
                if k == "ATE_T":
                    output_table[-1].append(f"{v:.3f}m")
                elif k == "ATE_rot":
                    output_table[-1].append(f"{v*57.3:.3f}°")
                else:
                    output_table[-1].append(v)
            if k == sort_index:
                sort_values.append(v)
    output_table = np.array(output_table)
    sort_values = np.array(sort_values)
    output_table = output_table[sort_values.argsort()]
    row0 = []
    for key in res[0]:
        row0.append(key)
    output_table = output_table.tolist()
    output_table.insert(0, row0)
    return tabulate.tabulate(output_table, tablefmt='html')

def grid_search(input_data, output_data, gt_data, agent_num, params, cmd="roslaunch d2pgo d2pgo_test_multi.launch ", is_root=True):
    #Gird search in param lists
    # print("Searching ", params)
    pg_gt = PoseGraph()
    pg_gt.read_g2o_single(gt_data)
    params_num = len(params)
    #DFS on params
    unique, row = check_param_unique(params)
    if unique:
        #Only one param, no need to grid search
        call_d2pgo(input_data, output_data, agent_num, params, cmd)
        pg_output = PoseGraph()
        pg_output.read_g2o_folder(output_data, prt=False)
        ate_T, ate_rot = ATE(pg_gt, pg_output)
        # print(params, ate_T, ate_rot)
        ret_data = {}
        for k, v in params:
            ret_data[k] = v[0]
        ret_data["ATE_T"] = ate_T
        ret_data["ATE_rot"] = ate_rot
        return [ret_data]
    else:
        #More than one param, need to grid search
        param_values = params[row][1]
        res = []
        for i in range(len(param_values)):
            new_params = copy.deepcopy(params)
            new_params[row][1] = [param_values[i]]
            ret = grid_search(input_data, output_data, gt_data, agent_num, new_params, cmd, is_root=False)
            for obj in ret:
                res.append(obj)
    return res

In [83]:
intput_data = "/home/xuhao/data/pgo/parking-garage/input"
gt_data = "/home/xuhao/data/pgo/parking-garage/groundtruth.g2o"
output_data = "/home/xuhao/data/pgo/parking-garage/output/"
agent_num = 5
params = [
    ["max_steps", [10]],
    ["rho_frame_T", np.logspace(-4, 0, 10, base=10)],
    ["rho_frame_theta", np.logspace(-3, 0, 10, base=10)]
]
ret = grid_search(intput_data, output_data, gt_data, agent_num, params)
print_res_to_table(ret)

max_steps,rho_frame_T,rho_frame_theta,ATE_T,ATE_rot
10,0.005994842503189409,0.021544346900318832,1.199m,0.698°
10,0.016681005372000592,0.46415888336127775,1.245m,0.858°
10,0.002154434690031882,0.21544346900318823,1.247m,0.594°
10,0.005994842503189409,0.01,1.277m,0.729°
10,0.005994842503189409,0.1,1.342m,0.741°
10,0.016681005372000592,0.01,1.356m,1.110°
10,0.005994842503189409,0.046415888336127774,1.358m,0.708°
10,0.005994842503189409,0.21544346900318823,1.363m,0.929°
10,0.016681005372000592,0.046415888336127774,1.372m,1.240°
10,0.005994842503189409,0.001,1.386m,0.718°


In [8]:
intput_data = "/home/xuhao/data/d2slam/pgo/tum_corr/input"
gt_data = "/home/xuhao/data/d2slam/pgo/tum_corr/ceres-output.g2o"
output_data = "/home/xuhao/data/d2slam/pgo/tum_corr/output/"
agent_num = 2
params = [
    ["max_steps", [100]],
    ["rho_frame_T", np.logspace(-4, 1, 10, base=10)],
    ["rho_frame_theta", np.logspace(-3, 1, 10, base=10)]
]
ret = grid_search(intput_data, output_data, gt_data, agent_num, params)
print_res_to_table(ret)

max_steps,rho_frame_T,rho_frame_theta,ATE_T,ATE_rot
100,0.0001,1.2915496650148828,0.787m,27.353°
100,0.0001,0.1668100537200059,0.788m,27.903°
100,0.0001,0.46415888336127775,0.794m,27.306°
100,0.0001,0.05994842503189409,0.796m,28.478°
100,0.0001,3.593813663804626,0.939m,29.287°
100,0.0001,10.0,1.188m,31.021°
100,0.004641588833612782,0.1668100537200059,1.283m,33.373°
100,2.782559402207126,1.2915496650148828,1.322m,30.843°
100,2.782559402207126,0.05994842503189409,1.372m,30.510°
100,2.782559402207126,0.46415888336127775,1.377m,30.821°
